# Ayudantía 06: Threading

## Autores: [@jfuentesg26](https://github.com/) & [@javi-saavedra](https://github.com/javi-saavedra) 


Recuerda que puedes evaluar la ayudantía en [este link](https://forms.gle/Udw8PNXGuNUB4CPS9)

# Threads 😏
Unidades pequeñas que pueden ser programadas para ser ejecutadas en un sistema operativo. 
Los "hilos" de un mismo proceso comparten memoria y estado de variables esto permite que se ejecuten mas rápido 

# Ilusión o realidad 😨
La verdad es que se simula la ejecucación en paralelo, lo que de verdad pasa es un **thread scheduling** o **time slicing**, esto es:

1.- Escoger un thread en espera

2.- Ejecutar algunas instrucciones de él

3.- Dejarlo en espera

4.- Volver al paso 1


# ¿Cómo crear threads?
Primero debemos importar la librería **threading** que nos permite crear nuestros propios "hilos" con la clase **Thread** 

In [0]:
import threading 

mi_hilo = threading.Thread(target = funcion, name = "HILO1")
mi_hilo.start()

Creamos una instancia de esta clase llamada mi_hilo, podemos entregarle un parámetro *target*, el cuál sera la función a ejecutar. Para poder iniciar el thread debemos utilizar **start()**.

Es importante recordar que este tipo de threads solo funciona **una vez**, para volver a usarlo hay que instanciarlo de nuevo.

Además tenemos el parámetro *name* que nos permite nombrar a nuestros threads y podemos saber cuál se esta ejecutanto con la siguiente función:

In [0]:
thread_actual = threading.current_thread()
nombre_thread = thread_actual.name

**¿Qué es el thread principal?**
El hilo que ejecuta el flujo principal del programa y todo proceso lo tiene, su nombre es **MainThread**



# ¡CUIDADO! 😱 *prints extraños*

Es probable que al ejecutar print falten o sobran saltos de línea. 

Esto ocurre por que (a nivel de máquina) escribir el texto del print y escribir el salto de línea son dos instrucciones distintas. Por lo tanto, es posible que un thread imprima el texto, se pause ese thread, luego otro thread imprima su texto y su respectivo salto de línea, y luego al volver al primer thread este imprima el salto de línea que faltaba.


# ¿Y si nuestra función recibe parámetros? 😲	
**No hay problema!** Utilizamos args y kwargs para darle los parámetros al thread, mira este ejemplo:

In [0]:
import threading
import time

def contar_ovejas_hasta(max_ovejas):
    thread_actual = threading.current_thread()
    print(f"{thread_actual.name} tiene sueño...")
    for numero in range(1, max_ovejas + 1):
        time.sleep(1)
        print(f"({thread_actual.name}: {numero} oveja{'s' if numero > 1 else ''})")
    print(f"{thread_actual.name} a dormir...")


# Se crean los threads usando la clase Thread, asociada a la función objetivo para 
# ser ejecutada por el thread, y los atributos de la función son ingresados en 
# args o kwargs

t1 = threading.Thread(name="Thread 1", target=contar_ovejas_hasta, args=(10,))
t2 = threading.Thread(name="Thread 2", target=contar_ovejas_hasta, kwargs={"max_ovejas": 15})
t1.start()
t2.start()

# ¿ Y si ocupamos OOP? 😍
**Que gran idea!** Podemos hacer clases para threads con comportamientos similares, esta clase debe heredar de la clase Thread. 
*¿Cómo lo hacemos* 

Debemos hacer un *__init__* en donde se hereda y hacer *override* al método **run** de la clase madre, para que cuando hagamos **start** este se ejecute y realiza las acciones que nosotros queramos :scream_cat:

Veamos un ejemplo:


In [0]:
import threading
import time
        
class CuentaLiebres(threading.Thread): # Hereda de Thread

    def __init__(self, nombre, max_liebres):
        super().__init__(name=nombre) 
        self.max_liebres = max_liebres
    
    def run(self):
        print(f"{self.name} tiene sueño...")
        tiempo_partida = time.time()
        for numero in range(1, self.max_liebres + 1):
            if numero % 2 == 1:
                time.sleep(1)
            print(f"({self.name}: {numero} liebre{'s' if numero > 1 else ''})")
        print(f"{self.name} a dormir...")
        print(f"{self.name} se durmió después de {time.time() - tiempo_partida} seg.")
        

# Se crean el thread
cuenta_liebres = CuentaLiebres("Antonio", 10)

# Se inicializa el thread creado
cuenta_liebres.start()


# **join()** ⏰

Si queremos que el programa principal espere a algun thread usamos éste método, despues de haber iniciado el hilo con start().

Podemos usar join(timeout = segundos) si queremos que el programa espere al thread por una cantidad definida de segundos o join(timeout = None) para que el programa espere al thread hasta que termine de ejecutarse.

El **thread que llama al método** queda bloqueado hasta que los threads referenciados terminen correctamente (o pase el tiempo establecido).


# **is_alive()** 🥳

Es posible identificar si un thread todavía está en funcionamiento mediante el uso del método is_alive(). Por lo general este método se implementa para saber el estado del thread después del uso de join(), al cual se le ha definido su tiempo máximo de espera.


### **Ahora veamos un ejemplo del uso de estos dos métodos**

En el siguiente ejemplo vemos como el *Main Thread* espera a los threads antonio y vicente. Además se comprueba si vicente sigue ejecutandose.

In [0]:

import threading
import time


# Usamos la definicion de los Thread declarados en el ejemplo anterior
# Se crean los threads usando la clase Thread.
antonio = CuentaLiebres("Antonio", 3)
vicente = CuentaLiebres("Vicente", 15)

# Se inicializan los threads creados
antonio.start()
vicente.start()
print("Ayudantes: Los profes se fueron a la cama...")

antonio.join()  # Esperaremos lo que sea necesario.
print("Ayudantes: ¡ANTONIO SE DURMIÓ!")
vicente.join(1)  # Esperaremos máximo 1 segundos después del último dormido, ya es muy tarde

if vicente.is_alive():
    print("Ayudantes: Vicente sigue despierto 😞. A la casa cabros.")
else:
    print("Ayudantes: ¡Todos los profes se durmieron! ¡A festejar!")
    for i in range(10):
        print("Ayudantes: 🎵🎶🎵🎶🎵🎶🎵🎶🎵🎶🎵🎶🎵🎶🎵🎶")
        time.sleep(1)

# **Daemons threads** 🙌
Anteriormente el programa ha esperado que terminen los threads para poder terminar. 

Con los *Daemons threads* no necesitamos preocuparnos de si terminaron o no, ya que cuando el programa principal termina, estos terminan automaticamente.

Así el programa termina cuando todos los threads no-daemons se hayan terminado.

Para identificar que los threads son de este tipo debemos poner en el constructor **daemon = True**.

Podemos hacer que el programa espere a un daemon thread con el método join, de la misma forma antes explicada. 

Un dato importante es que luego de iniciar un thread con start, no puedes cambiarlo de daemon thread a thread (o viceversa), ya que saldrá un error del tipo *RuntimeError*-

Veamos dos ejemplos, el primero llamando a la clase Thread directamente y el otro con OOP (recuerda que estos códigos tienen problemas corriendo en jupyter notebook así que es mejor que los pruebas desde tu consola)




In [0]:

import threading
import time


def dormilon():
    print(f"{threading.current_thread().name} tiene sueño...")
    time.sleep(2)
    print(f"{threading.current_thread().name} se durmió.")

    
def con_insonmio():
    print(f"{threading.current_thread().name} tiene sueño...")
    time.sleep(10)
    print(f"{threading.current_thread().name} se durmió.")


# Forma 1 de hacer un thread daemon
dormilon = threading.Thread(name="Dormilón", target=dormilon, daemon=True)
# Forma 2 de hacer un thread daemon
con_insomnio = threading.Thread(name="Con insonmio", target=con_insonmio)
con_insomnio.daemon = True

# Se inicializan los threads
dormilon.start()
con_insomnio.start()

In [0]:
import threading
class Daemon(threading.Thread):
    
    def __init__(self):
        super().__init__()
        # Cuando inicializamos el thread lo declaramos como daemon
        self.daemon = True
    
    def run(self):
        print("Daemon thread: Empezando...")
        time.sleep(2)
        print("Daemon thread: Terminando...")

daemon = Daemon()
daemon.start()
daemon.join()

# **Timers** ⌛️
Subclase de Thread, la cual al ser ejecutado espera un tiempo en segundos indicado y luego realiza las instrucciones determinadas. 

Poseen un método **cancel()** para cancelar su ejecución.

Se deben entregar como parámetros el tiempo en segundos, la función y los parametros requeridos por esta.


In [0]:
def mi_timer(ruta_archivo):
    with open(ruta_archivo) as archivo:
        for linea in archivo:
            print(linea)

t1 = threading.Timer(10.0, mi_timer, args=("files/mensaje_01.txt",))
t2 = threading.Timer(5.0, mi_timer, kwargs={"ruta_archivo": "files/mensaje_02.txt"})

t1.start() # el thread t comenzará después de 10 seconds
t2.start() # el thread t comenzará después de 5 seconds

# **Mecanismos de Sincronización** ⏱
Cuando más de un thread debe compartir el acceso a determinados recursos (archivos, variables, etc) de manera **concurrente** se nos pueden generar errores, ya que no sabemos como se entremezclan los threads ni tampoco cuando su operación es pausada. 

Es por esto que el código que sigue a continuación no entrega el resultado correcto: 

In [0]:
import threading

class Contador: 
    def __init__(self):
        self.valor = 0

        
def sumador(contador):
    for _ in range(10 ** 6):
        contador.valor += 1


contador = Contador()        
t1 = threading.Thread(target=sumador, args=(contador,))
t2 = threading.Thread(target=sumador, args=(contador,))

t1.start()
t2.start()
t1.join()
t2.join()

print("Listo, nuestro contador vale", contador.valor)

Esto ocurre ya que no nos aseguramos de que la operación sea **atómica**, es decir, que ningún thread la pueda iniciar a menos que ningún otro la esté haciendo. Al conjunto de instrucciones atómicas se le denomina sección crítica.

Para arreglar estos problemas tenemos los mecanismos de sincronización.

#  **Lock** 🔐
La clase **Lock** de la librería threading permite que haya un solo thread en una sección crítica a la vez.
Estos locks pueden estar desbloqueados (inicialmente) o bloqueados. 
La función **acquire()** permite adquirir el lock por parte de un thread y dejarlo bloqueado para los otros. 
Por su parte la función **release()** libera el lock (lo desbloquea) , quedando disponible para que cualquier thread pueda adquirirlo. 

A continuación un ejemplo: (*Recuerda correrlo en consola*)


In [0]:
import threading


lock_global = threading.Lock()

def sumador_con_seccion_critica(contador):
    for _ in range(10 ** 6):
        # Pedimos el lock antes de entrar a la sección crítica.
        lock_global.acquire()
        # --- Sección crítica ---. 
        # Está garantizado que en estas líneas sólo habrá un thread a la vez.
        contador.valor += 1
        # --- Fin de la sección crítica ---.
        # Liberamos el lock luego de salir de la sección crítica.
        lock_global.release()
contador = Contador()        
t1 = threading.Thread(target=sumador_con_seccion_critica, args=(contador,))
t2 = threading.Thread(target=sumador_con_seccion_critica, args=(contador,))

t1.start()
t2.start()
t1.join()
t2.join()

print("Listo, nuestro contador vale", contador.valor)

Otra forma de hacerlo sin usar release y acquire es con **with** como muestra el ejemplo:

In [0]:
import time


lock_global = threading.Lock()

def sumador(contador):
    nombre = threading.current_thread().name
    for _ in range(10):
        with lock_global:
            # --- Sección crítica ---. 
            # Está garantizado que en estas líneas sólo habrá un thread a la vez.
            valor = contador.valor
            print(f"{nombre}: lee {valor}")
            nuevo_valor = valor + 1
            print(f"{nombre}: suma 1 => {nuevo_valor}")
            contador.valor = nuevo_valor
            print(f"{nombre}: guarda {nuevo_valor}")
            time.sleep(1)
            # --- Fin de la sección crítica ---.

contador = Contador()        
t1 = threading.Thread(name="T1", target=sumador, args=(contador,))
t2 = threading.Thread(name="T2", target=sumador, args=(contador,))

t1.start()
t2.start()
t1.join()
t2.join()

# **Señales entre threads** 📡
Nos ayudan cuando queremos que el thread espere hasta que cierto suceso ocurra. 

Para esto recibimos ayuda de los objetos **Event**. 

¿Cómo funciona? Un thread hace la señal y otros lo esperan. 

Los *Event* poseen un flag interno, el cual toma valor *True* cuando la señal esta activa y *False* cuando no.

Para que los thread esperen la señal utilizamos el método **wait()** y para hacer la señal llamamos al método **set()** lo que deja el *flag del Event* en *True*. Si queremos resetear la señal usamos **clear()** para hacer el flag igual a *False*.

Veamos un ejemplo:

In [0]:
# Ejemplo sacado de http://zulko.github.io/blog/2013/09/19/a-basic-example-of-threads-synchronization-in-python/

import threading
import time


# Tenemos dos eventos o señales.
# Esta es para avisar que el video ya está listo para ser reproducido.
video_cargado = threading.Event()
# Esta es para avisar que el audio ya está listo para ser reproducido.
audio_cargado = threading.Event()

def reproducir_video(nombre):
    print(f"Cargando video {nombre} en t={time.time():.6f}")
    # Supongamos que se demora 3 segundos
    time.sleep(3)
    print(f"¡Video cargado! en t={time.time():.6f}")
    # Avisamos que el video ya está cargado
    video_cargado.set()
    # Esperamos a que el audio ya se haya cargado
    audio_cargado.wait()
    # ¡Listo!
    print(f"Reproduciendo video en t={time.time():.6f}")
    
    
def reproducir_audio(nombre):
    print(f"Cargando audio {nombre} en t={time.time():.6f}")
    # Supongamos que se demora 5 segundos
    time.sleep(5)
    print(f"¡Audio cargado! en t={time.time():.6f}")
    # Avisamos que el audio ya está cargado
    audio_cargado.set()
    # Esperamos a que el video ya se haya cargado
    video_cargado.wait()
    # ¡Listo!
    print(f"Reproduciendo audio en t={time.time():.6f}")
    
    
t1 = threading.Thread(target=reproducir_audio, args=("'No Te Enamores' - Paloma Mami",))
t2 = threading.Thread(target=reproducir_video, args=("'No Te Enamores' - Paloma Mami",))

t1.start()
t2.start()

t1.join()
t2.join()

# **DeadLocks** 🧛‍ 🔒

También llamado interbloqueo hace referencia al error en donde dos threads se esperan mutuamente, por lo que ninguno de los dos avanza.

Por lo que al programar debes tener cuidado para que esto no pase.

# **Ejercicio Propuesto**  👩🏾‍💻
Leyendo por internet, has encontrado el trabajo perfecto para ti. Te han contratado para simular la recolección de materiales de la DCCueva. Aquí trabajan mineros pero de manera poco óptima, perdiendo DCCriptoMonedas y provocando grandes pérdidas para el DCC.

En tu primer día, se te entrega el código base que han estado utilizando para simular la recolección dentro de la DCCueva:

In [0]:
import time
import random

class Minero:

    def __init__(self, nombre):
        self.nombre = nombre
        self.velocidad = random.randint(2, 4)
        self.cantidad = 0
        self.adentro = False

    def recolectar_recursos(self):
        cantidad = random.randint(5, 15)
        tiempo = cantidad/self.velocidad
        self.adentro = True
        time.sleep(tiempo)
        print(f'Trabajador {self.nombre} ha recolectado {cantidad} DCCriptoMonedas')
        self.cantidad += cantidad
        self.adentro = False

    def trabajar(self):
        for i in range(3):
            print(f'Trabajador {self.nombre} ha entrado a la DCCueva')
            self.recolectar_recursos()


t1 = Minero('John')
t2 = Minero('Alex')
t3 = Minero('Peter')

t1.trabajar()
t2.trabajar()
t3.trabajar()

total = t1.cantidad + t2.cantidad + t3.cantidad
print('------------------------------------------')
print(f'Se han recolectado {total} DCCriptoMonedas')

Modela el ejercicio anterior de modo que ahora el programa se ejecute de manera concurrente (Threads).

In [0]:
import time
import random
from threading import Thread

#Implementar modelacion con Thread
class Minero():

    def __init__(self, nombre):
        #Completar clase

    def recolectar_recursos(self):
        cantidad = random.randint(5, 15)
        tiempo = cantidad/self.velocidad
        self.adentro = True
        time.sleep(tiempo)
        print(f'Trabajador {self.nombre} ha recolectado {cantidad} DCCriptoMonedas')
        self.cantidad += cantidad
        self.adentro = False

    def trabajar(self): #Puedes modificarlo si quieres trabajar con herencia ;)
        #Completar metodo


t1 = Minero('John') #Eres libre de modificar los nombres
t2 = Minero('Alex') #Eres libre de modificar los nombres
t3 = Minero('Peter') #Eres libre de modificar los nombres :)

t1.start()
t2.start()
t3.start()

total = t1.cantidad + t2.cantidad + t3.cantidad
print('------------------------------------------')
print(f'Se han recolectado {total} DCCriptoMonedas')

# **Actividad** 🎉

Bajo el contexto nacional las ayudantes que prepararon esta ayudantía (**Las Javis**) se pusieron de acuerdo para que una de ellas revisara y preparara el contenido de la ayudantía de *Threading*, mientras que otra prepara el ejercicio que se realizaría en esta. Cuando alguna de ellas termina de preparar el contenido o ejercicio de una temática específica de la ayudantía debe escribirlo en *DCCuaderno Virtual*. Sin embargo, se ha hayado un gran problema en la plataforma, las ayudantes **no pueden escribir al mismo tiempo** dentro de este y cada una demora un tiempo específico en escribir lo preparado, entonces, si una ayudante está escribiendo la otra debe esperar ese tiempo para escribir su parte. 

Las Javis deben terminar lo más pronto posible la ayudantía, por lo que solicitan tu ayuda para terminarla de una vez por todas 😔👌. 

Para llevar a cabo lo anterior se te entrega la clase ```Contenido``` que ya viene implementada, y la clase ```Ayudante``` la cual debes completar. También se te entrega la función ```cargar_contenidos``` que carga los csv de los contenidos a repasar en la ayudantía con los respectivos tiempos que toma la ayudante en escribirlos en *DCCuaderno Virtual*.

In [0]:
import threading
import time

class Contenido:

    def __init__(self, tema, tiempo):
        self.tema = tema
        self.tiempo = tiempo


In [0]:
class Ayudante(threading.Thread):
    
    #Completar

    def __init__(self, nombre, contenidos, funcion):

        super().__init__()
        self.nombre = nombre
        self.contenidos = contenidos #lista de contenidos
        self.funcion = funcion #contenido o ejercicio

    def run(self):
        while self.contenidos:
            contenido = self.contenidos.pop(0)
            #Completar
        
        print(f"{self.nombre}: Ayudantía lista!")


Dado que las ayudantes son muy unidas se van a esperar mutuamente antes de dar la ayudantía por finalizada, entonces, si una ya terminó su parte debe esperar a que la otra termine.👩‍❤️‍👩

In [0]:
class Ayudante(threading.Thread):

    lock_escribir = threading.Lock() 
    #Completar

    def __init__(self, nombre, contenidos, funcion):

        super().__init__()
        self.nombre = nombre
        self.contenidos = contenidos #lista de contenidos
        self.funcion = funcion #contenido o ejercicio

    def run(self):
        while self.contenidos:
            contenido = self.contenidos.pop(0)
            time.sleep(1)
            with self.lock_escribir:
                print(f'{self.nombre}: Estoy escribiendo el {self.funcion} del tema {contenido.tema}')
                time.sleep(contenido.tiempo)
        #Completar

In [0]:
def cargar_contenidos(path):
    contenidos = []

    with open(path, 'r', encoding='utf-8') as file:
        lineas = file.readlines()
        for linea in lineas:
            tema, tiempo = linea.strip().split(",")
            contenido = Contenido(tema, int(tiempo))
            contenidos.append(contenido)

    return contenidos

In [0]:
contenidos_1 = cargar_contenidos('contenidos.csv')
contenidos_2 = cargar_contenidos('contenidos_ewe.csv')

javi_fuentes = Ayudante("Javi Fuentes", contenidos_1, "contenido")
javi_ewe = Ayudante("Javi ewe", contenidos_2, "ejercicio")

javi_ewe.start()
javi_fuentes.start()

javi_fuentes.join()
javi_ewe.join()

print("La ayudantía está terminada :)")

La ayudantía ya está lista! Solo falta que la **revisen los profesores** y el gran *Dr. Pinto* para obtener la aprobación final y poder enseñársela a los estudiantes ~~estresados~~ que quieren aprender. Los profesores tienen un tiempo límite para entregar su feedback, si dentro de ese tiempo no alcanzan a revisar la ayudantía o dar sus comentarios al respecto, simplemente no se tomarán en cuenta.

Para esto se te entrega la clase ```Reloj``` que ya viene implementada, y la clase ```Profesor``` que debes completar para llevar a cabo tu misión 😎. 

In [0]:
class Reloj(threading.Thread):
    def __init__(self, tiempo):
        super().__init__()
        self.tiempo = tiempo

    def run(self):
        while self.tiempo > 0:
            print(f'Tiempo restante: {self.tiempo}')
            time.sleep(1)
            self.tiempo -= 1




In [0]:
class Profesor(threading.Thread):
    
    #Completar

    def __init__(self, nombre, revision):
        super().__init__()
        self.nombre = nombre
        self.revision = revision
        #Completar
        
    def run(self):
        #Completar
        print('-------------------------------------------')
        print(f'{self.nombre}: {self.revision["feedback"]}')
        print('-------------------------------------------')

In [0]:
def cargar_revisiones(path):
    revisiones = dict()

    with open(path, 'r', encoding='utf-8') as file:
        lineas = file.readlines()
        for linea in lineas:
            nombre, feedback, tiempo = linea.strip().split(";")
            revisiones[nombre] = {"feedback": feedback, "tiempo": int(tiempo)}

    return revisiones

In [0]:
revision = cargar_revisiones('revisiones.csv')

vicente = Profesor("Vicente", revision["Vicente"])
fernando = Profesor("Fernando", revision["Fernando"])
nini = Profesor("Nini", revision["Nini"])
cristian = Profesor("Cristian", revision["Cristian"])

reloj = Reloj(30)
reloj.start()

vicente.start()
fernando.start()
nini.start()
cristian.start()

reloj.join()